### Content

### References

[Bài 3: Conditional GAN (cGAN)](https://nttuan8.com/bai-3-conditional-gan-cgan/)

[Bài 4: Least Squares Generative Adversarial Networks (LSGAN)]()

[Bài 5: GAN evaluation](https://nttuan8.com/bai-4-least-squares-generative-adversarial-networks-lsgan/)